# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links, @domainnames.com,mailto

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [12]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links, @domainnames.com,mailto

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic

In [13]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [14]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company / Nebula portfolio',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [15]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [16]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.d

In [17]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'social media', 'url': 'https://twitter.com/huggingface'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [18]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [19]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
about 12 hours ago
•
2.07k
•
758
tencent/HunyuanVideo-1.5
Updated
1 day ago
•
2.12k
•
740
black-forest-labs/FLUX.2-dev
Updated
about 13 hours ago
•
116k
•
643
tencent/HunyuanOCR
Updated
about 11 hours ago
•
24.9k
•
458
facebook/sam3
Updated
7 days ago
•
182k
•
738
Browse 1M+ models
Spaces
Running
on
Zero
MCP
310
Z Image Turbo
🏃
310
Generate images from text prompts
Running
on
Zero
212
FLUX.2 [dev]
💻
212
Generate detailed images from text prompts or edit existing images
Running
on
Zero
MCP
Featured
1.31

In [20]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\nabout 12 hours ago\n•\n2.07k\n•\n758\ntencent/HunyuanVideo-1.5\nUpdated\n1 day ago\n•\n2.12k\n•\n740\nblack-forest-labs/FLUX.2-dev\nUpdated\nabout 14 hours ago\n•\n116k\n•\n643\ntencent/HunyuanOCR\nUpdated\nabout 11 hours ago\n•\n24.9k\n•\n458\nfacebook/sam3\nUpdated\n7 days ago\n•\n182k\n•\n738\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\n310\nZ Im

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a vibrant AI community and collaboration platform that is pioneering the future of machine learning. Serving as a central hub, Hugging Face empowers machine learning engineers, data scientists, researchers, and developers by providing an open and ethical environment to share, explore, and experiment with models, datasets, and applications.

With a rapidly expanding community, Hugging Face hosts over **1 million models**, **250,000 datasets**, and **400,000+ applications**, making it one of the largest and most inclusive ecosystems for machine learning innovation.

---

## What We Offer

### Open Collaboration Platform
- Host, share, and collaborate on **unlimited public models, datasets, and applications**.
- Access and contribute to cutting-edge machine learning technologies across **text, image, video, audio, and even 3D modalities**.
- Build and showcase your machine learning portfolio and gain recognition in the global AI community.

### State-of-the-Art Resources
- Browse and experiment with a massive library of open-source machine learning models and datasets from top contributors including industry leaders and research labs.
- Engage with trending and featured models such as Tongyi-MAI/Z-Image-Turbo, Tencent/HunyuanVideo, Facebook/SAM3, and many more.

### Compute and Enterprise Solutions
- Accelerate your machine learning workflows with paid **compute resources** optimized for training and inference.
- Dedicated **Enterprise** offerings to support teams and organizations with enhanced collaboration, security, and scalability.

---

## Company Culture

- **Community-Driven**: At its core, Hugging Face is about community, learning, and mutual support. The platform fosters a collaborative spirit where ideas and knowledge flow freely.
- **Open & Ethical AI**: Committed to building an open AI ecosystem that values transparency, fairness, and responsibility.
- **Innovative & Inclusive**: Hugging Face embraces diversity in thought and background, inviting contributors worldwide to push the boundaries of machine learning.
- **Continuous Learning**: Providing extensive documentation, resources, and spaces to empower individuals at all skill levels—from beginners to experts.

---

## Our Customers

Hugging Face serves a diverse user base that includes:

- **Machine Learning Engineers & Data Scientists** looking to accelerate development cycles.
- **Academic and Research Institutions** leveraging open datasets and models for experimentation.
- **Startups and Enterprises** integrating AI into their products via robust, scalable solutions.
- **AI Enthusiasts and Hobbyists** eager to learn, create, and share in a vibrant community.

Partners and contributors come from renowned organizations like **NVIDIA**, **Tencent**, **Facebook**, and many others who trust Hugging Face for cutting-edge AI collaboration.

---

## Careers at Hugging Face

Join Hugging Face if you are passionate about:

- Pioneering open-source AI projects that impact millions across the globe.
- Working alongside a global, multidisciplinary team dedicated to ethical and innovative technology.
- Contributing to a mission-driven company that values transparency, collaboration, and social impact.

Open roles typically include positions in:
- Machine Learning Research and Engineering
- Software Development
- Community Management and Developer Relations
- Product and Enterprise Solutions

Visit Hugging Face’s careers page to explore opportunities and become part of building the future of AI.

---

## Get Involved

- **Explore and use models**: Dive into more than 1 million pre-trained models tailored for tasks across multiple AI domains.
- **Share your work**: Upload your models, datasets, and applications to the Hub and join a global network of innovators.
- **Learn and grow**: Access comprehensive documentation and community resources to boost your AI skills.

**Join the AI community building the future.**

[Sign Up on Hugging Face](https://huggingface.co/signup)

---

## Contact & Resources

- Website: https://huggingface.co
- Community Forums & Discussions
- Extensive Documentation & Tutorials
- Enterprise and Support Services

---

*Hugging Face – The AI community building the future.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a vibrant AI community dedicated to building the future of machine learning. It serves as a collaborative platform where developers, researchers, and enterprises around the world come together to create, share, and discover state-of-the-art machine learning models, datasets, and applications.

With over 1 million models and 250,000 datasets hosted, Hugging Face is the home for anyone passionate about advancing AI technology through open-source collaboration and innovation.

---

## What We Offer

- **Models**  
  Access and contribute to a vast library of over 1 million pre-trained machine learning models across multiple modalities including text, image, video, audio, and even 3D.

- **Datasets**  
  Explore a rich collection of over 250,000 datasets to power your machine learning experiments and applications.

- **Spaces**  
  Run and share machine learning applications seamlessly. This feature promotes community engagement by hosting interactive AI demos and tools.

- **Open Source Tools**  
  Utilize the Hugging Face open-source stack designed to accelerate your ML development workflow.

- **Enterprise Solutions**  
  Get scalable, secure compute resources and team collaboration features tailored for business needs.

---

## Company Culture

Hugging Face fosters an open and ethical AI community focused on inclusivity, collaboration, and transparency. It empowers a diverse global network of machine learning engineers, scientists, and enthusiasts by:

- Promoting open-source sharing and collaborative development
- Supporting ethical practices in AI research and deployment
- Encouraging continuous learning and skill-building through sharing knowledge and best practices

The company culture values innovation, community-driven growth, and building responsible AI technologies that benefit everyone.

---

## Key Customers and Users

Hugging Face serves a broad spectrum of customers including:

- Independent AI researchers and developers building cutting-edge machine learning models
- Large enterprises deploying machine learning applications at scale
- Academic institutions advancing AI research
- AI enthusiasts and hobbyists eager to learn and contribute to the AI revolution

Notably, industry leaders like NVIDIA, Tencent, and Facebook contribute models and datasets, showcasing Hugging Face’s strong adoption across major tech players.

---

## Careers & Opportunities

Hugging Face is growing rapidly and seeks talented individuals passionate about AI and open-source technology. Career opportunities include roles in:

- Machine Learning Engineering
- Research and Development
- Software Engineering
- Community and Developer Relations
- Enterprise Solutions and Customer Success

Joining Hugging Face means becoming part of an innovative, fast-paced environment committed to shaping the future of AI through collaboration and ethical frameworks.

---

## Get Involved

- **Join the Community:** Sign up to share your models, datasets, or applications and collaborate with other AI innovators.  
- **Explore AI Applications:** Discover and experiment with thousands of live machine learning applications via Hugging Face Spaces.  
- **Accelerate with Enterprise:** Take advantage of premium compute solutions tailored for teams to streamline AI deployment.

Visit [huggingface.co](https://huggingface.co) to learn more and start your AI journey with the global machine learning community building tomorrow’s technology today.

---

## Brand Highlights

- Signature Colors: Vibrant yellow (#FFD21E and #FF9D00) coupled with modern gray (#6B7280) reflect the company’s energetic and trustworthy brand identity.
- Logo and assets are freely available for those engaging with the Hugging Face community and ecosystem.

---

*Hugging Face*  
*The AI community building the future together.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>